In [2]:
import os
import sys

spark_path = "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import udf

import datetime as dt
import pandas as pd

sc = SparkContext("local")
sqlCtx = SQLContext(sc)

print sc
print sqlCtx

In [10]:
x = sc.parallelize([('B',['A',1]),('B',['C',2]),('A',['C',3]),('A',['B',4]),('A',['D',5])])
#xprime = x.map(lambda r: [(r[0], (r[1], r[2]))])
x.take(5)

[('B', ['A', 1]),
 ('B', ['C', 2]),
 ('A', ['C', 3]),
 ('A', ['B', 4]),
 ('A', ['D', 5])]

In [7]:


zeroVal = []
mergeVal = (lambda aggregated, el: aggregated + [(el)])
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2
y = x.aggregateByKey(zeroVal,mergeVal,mergeComb)
print(x.collect())
print(y.collect())


[('B', ['A', 1]), ('B', ['C', 2]), ('A', ['C', 3]), ('A', ['B', 4]), ('A', ['D', 5])]
[('A', [['C', 3], ['B', 4], ['D', 5]]), ('B', [['A', 1], ['C', 2]])]


In [27]:
x = sc.parallelize([('B','A',21),('B','C',12),('A','C',3),('A','B',2),('A','D',5)])
xprime = x.map(lambda r: (r[0], [r[1], r[2]]))
xprime.take(5)

[('B', ['A', 21]),
 ('B', ['C', 12]),
 ('A', ['C', 3]),
 ('A', ['B', 2]),
 ('A', ['D', 5])]

In [70]:
zeroVal = []
mergeVal = (lambda aggregated, el: aggregated + [(el)])
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2
y = xprime.aggregateByKey(zeroVal,mergeVal,mergeComb)
print(xprime.collect())
print(y.collect())

# assuming it's sorted
# take top k
calls_k = 2
takeTopK = (lambda r: (r[0], [i for i in r[1][1:calls_k]]))
y = y.map(takeTopK)
print(y.collect())

[('B', ['A', 21]), ('B', ['C', 12]), ('A', ['C', 3]), ('A', ['B', 2]), ('A', ['D', 5])]
[('A', [['C', 3], ['B', 2], ['D', 5]]), ('B', [['A', 21], ['C', 12]])]
[('A', [['B', 2]]), ('B', [['C', 12]])]


In [71]:
# remove counts
y = y.map(lambda r: [r[0], [i[0] for i in r[1]]])
print(y.collect())

# convert to dataframe
y_df = y.toDF(["a_imsi","callingcir"])
y_df.show()

[['A', ['B']], ['B', ['C']]]
+------+----------+
|a_imsi|callingcir|
+------+----------+
|     A|       [B]|
|     B|       [C]|
+------+----------+



In [50]:
y = y.sortBy(lambda r: x[1] for x in r)
y.collect()
y.take(5)

NameError: name 'r' is not defined

In [48]:
def getCount(r): return (x[1] for x in r[1])

foo = y.map(getCount)

In [49]:
foo.count()
type(foo)

pyspark.rdd.PipelinedRDD